In [1]:
import pandas as pd
import re

# Charger le fichier CSV
df = pd.read_csv('filtered_tweets_engie.csv', delimiter=';')

# Convertir les identifiants (id) en chaînes de caractères (assurer le bon format)
df['id'] = df['id'].astype(str)

# Convertir la colonne 'created_at' en datetime
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')

# Supprimer la colonne 'name' (expliqué dans la consigne)
df = df.drop(columns=['name'])

# Supprimer les tweets où 'screen_name' est 'ENGIEpartFR'
df = df[df['screen_name'] != 'ENGIEpartFR']

# Supprimer les caractères spéciaux, les emojis et les sauts de ligne
def clean_text(text):
    # Enlever les sauts de ligne et les retours chariots
    text = text.replace('\n', ' ').replace('\r', ' ')
    # Enlever les emojis avec une expression régulière
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Enlever les caractères non-ASCII (comme les emojis)
    # Enlever les caractères spéciaux
    text = re.sub(r'[^\w\s,!?;.]', '', text)  # Garde seulement les lettres, chiffres et certains signes de ponctuation
    return text

df['full_text'] = df['full_text'].apply(clean_text)

# Supprimer les messages successifs du même 'screen_name'
df['prev_screen_name'] = df['screen_name'].shift(1)  # Créer une colonne pour le 'screen_name' précédent
df['is_duplicate'] = df['screen_name'] == df['prev_screen_name']  # Marquer les messages successifs

# Filtrer les doublons successifs
df_filtered = df[~df['is_duplicate']]  # Garder uniquement les tweets qui ne sont pas des doublons successifs

# Supprimer la colonne temporaire
df_filtered.drop(columns=['prev_screen_name', 'is_duplicate'], inplace=True)

# Sauvegarder le DataFrame nettoyé dans un nouveau fichier CSV
df_filtered.to_csv('filtered_tweets_engie_cleaned.csv', index=False, sep=';')

# Afficher les premières lignes du DataFrame nettoyé
print(df_filtered.head())


            id      screen_name                 created_at  \
0  1,72517E+18       gptournier  2023-11-16 16:13:18+01:00   
1  1,72868E+18       jouanetwan  2023-11-26 08:34:34+01:00   
2  1,73784E+18  vince_thouvenin  2023-12-21 15:27:08+01:00   
3  1,74049E+18   BiduleAnatheme  2023-12-28 22:32:58+01:00   
4  1,74068E+18  vince_thouvenin  2023-12-29 11:08:10+01:00   

                                           full_text  
0  ENGIEpartFR n6 mois d attente et tjs aucune r ...  
1  Bonjour ENGIEpartSAV , l appli monpilotageelec...  
2  ENGIEpartFR mon syndic de copropri t  sergic e...  
3  ENGIEpartSAV vous envisagez de vous occuper de...  
4  ENGIEpartSAV retour de votre technicien vous n...  


C:\Users\baayv\AppData\Local\Temp\ipykernel_4636\590267225.py:11: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
C:\Users\baayv\AppData\Local\Temp\ipykernel_4636\590267225.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['prev_screen_name', 'is_duplicate'], inplace=True)
